In [224]:
import os
import pathlib

import torch
import torchsummary # 모델 구조 보기좋게 출력
import torchvision
from torchvision import utils
from torchvision import models
from torchvision import transforms
from torchvision.utils import *
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
from torch import optim
from torch.nn import *

import numpy as np
from pathlib import Path

##### device 세팅

In [225]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

##### datasets
* STL10 사용
* download 가능

[Const]
* DATA_NUM = int
* BATCH_SIZE = int
* BATCH_NUM = int = DATA_NUM / BATCH_NUM

[Data Structure]
* Dataset = Sample[DATA_NUM]
* Sample = (Image, Label)
* Image = Tensor[width,height,channel]
* Label = Tensor[1]
* DataLader = Batch[batch_num]
* Batch = (ImageBatch, LabelBatch)
* ImageBatch = Image[batch_size]
* LabelBatch = Label[batch_size]

In [226]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
size = [64, 64]

torchvision.transforms.Compose([
    torchvision.transforms.Resize(size),
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Normalize(mean, std),
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.STL10('/train', split='train', download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.STL10('/test', split='test', download=True, transform=transforms.ToTensor())

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


##### Model
* torchvision.models 에서 여러 모델 로드 가능
* pretrained weight도 다운 가능

In [227]:
# pretrined weight 받아오기
pretrained_weight = models.ResNet50_Weights.IMAGENET1K_V2
# weight을 주고 resnet 생성
model = models.resnet50(weights = pretrained_weight).to(device)
# 모델 뒷단의 FC를 output_size가 10인 FC로 수정
model.fc = Linear(in_features=2048, out_features=10, bias=True).to(device)

##### Training Parameter

In [233]:
params = {
    'num_epochs':5,
    'optimizer':optim.Adam(model.parameters(), lr=0.000001),
    'loss_function':nn.CrossEntropyLoss().to(device),
    'train_dataloader':train_dataloader,
    'test_dataloader': test_dataloader,
    'device':device
}

##### Training

In [234]:
def train(model, params):
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    test_dataloader=params["test_dataloader"]
    device=params["device"]
    optimizer = params["optimizer"]
    
    for epoch in range(0, num_epochs):
        for i, data in enumerate(train_dataloader, 0):
            # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 이전 batch에서 계산된 가중치를 초기화
            optimizer.zero_grad() 

            # forward + back propagation 연산
            outputs = model(inputs)
            train_loss = loss_function(outputs, labels)
            train_loss.backward()
            optimizer.step()

        # test accuracy 계산
        total = 0
        correct = 0
        accuracy = []
        
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 결과값 연산
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss = loss_function(outputs, labels).item()
            accuracy.append(100 * correct/total)

        # 학습 결과 출력
        print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f' %(epoch+1, num_epochs, train_loss.item(), test_loss, 100*correct/total))

In [235]:
train(model, params)

Epoch: 1/5, Train loss: 0.166574, Test loss: 0.190729, Accuracy: 92.67
Epoch: 2/5, Train loss: 0.328720, Test loss: 0.192077, Accuracy: 92.72
Epoch: 3/5, Train loss: 0.600106, Test loss: 0.192423, Accuracy: 92.71
Epoch: 4/5, Train loss: 0.120998, Test loss: 0.193277, Accuracy: 92.69
Epoch: 5/5, Train loss: 0.059825, Test loss: 0.193585, Accuracy: 92.65
